In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import polars as pl
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
FINAL_MODEL = None
SCALER = None
FEATURE_COLUMNS = []
HISTORY_BUFFER = None
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEQUENCE_LENGTH = 20

In [ ]:
 class LSTMPredictor(nn.Module):
   """LSTM-based model for market return prediction"""
   def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.2):
        super(LSTMPredictor, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.Tanh(),
            nn.Linear(hidden_dim // 2, 1)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 1)
        )
   def forward(self, x):
        lstm_out, (hidden, cell) = self.lstm(x)
        attention_weights = torch.softmax(self.attention(lstm_out), dim=1)
        context = torch.sum(attention_weights * lstm_out, dim=1)        
        output = self.fc(context)
        return output

In [ ]:
def convert_to_signal(predictions: np.ndarray, multiplier: float = 400.0) -> np.ndarray:
    signals = predictions * multiplier + 1
    return np.clip(signals, 0.0, 2.0)

def simple_impute(df):
    df_imp = df.copy()
    
    numeric_cols = df_imp.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df_imp[col] = df_imp[col].fillna(method='ffill').fillna(method='bfill').fillna(0)
    
    return df_imp

In [ ]:
def create_sequences(X, y, sequence_length):
    """Create sequences for LSTM training"""
    X_seq = []
    y_seq = []
    
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:i + sequence_length])
        y_seq.append(y[i + sequence_length])
    
    return np.array(X_seq), np.array(y_seq)

In [ ]:
def train_model():
    global FINAL_MODEL, SCALER, FEATURE_COLUMNS, HISTORY_BUFFER, SEQUENCE_LENGTH
    
    print("Training LSTM model...")
    print(f"Using device: {DEVICE}")
    start_time = pd.Timestamp.now()
    
    path = "/kaggle/input/hull-tactical-market-prediction"
    print(f"Loading data from {path}")
    
    training_df = pd.read_csv(f"{path}/train.csv")
    print(f"Loaded {len(training_df)} rows")
    
    print("Imputing missing values...")
    training_df = simple_impute(training_df)
    print(f"Imputation done. Remaining NaN: {training_df.isnull().sum().sum()}")
    
    if isinstance(training_df, pd.DataFrame):
        training_df = pl.from_pandas(training_df)
    
    training_df = training_df.rename({'market_forward_excess_returns': 'target'})
    
    feature_cols = [col for col in training_df.columns if col != 'date_id']
    training_df = training_df.with_columns(pl.col(feature_cols).cast(pl.Float64, strict=False))
    
    training_df = training_df.drop_nulls()
    print(f"After dropping nulls: {len(training_df)} rows")
    
    FEATURE_COLUMNS = [col for col in training_df.columns 
                       if col not in ['date_id', 'target', 'forward_returns', 'risk_free_rate']]
    
    print(f"Training with {len(FEATURE_COLUMNS)} features")
    
    X_train = training_df.select(FEATURE_COLUMNS).to_numpy()
    y_train = training_df.select('target').to_numpy().ravel()
    
    SCALER = StandardScaler()
    X_train_scaled = SCALER.fit_transform(X_train)
    
    print(f"Creating sequences with length {SEQUENCE_LENGTH}...")
    X_seq, y_seq = create_sequences(X_train_scaled, y_train, SEQUENCE_LENGTH)
    print(f"Created {len(X_seq)} sequences")
    
    X_tensor = torch.FloatTensor(X_seq).to(DEVICE)
    y_tensor = torch.FloatTensor(y_seq).reshape(-1, 1).to(DEVICE)
    
    dataset = TensorDataset(X_tensor, y_tensor)
    train_loader = DataLoader(dataset, batch_size=128, shuffle=True)
    
    input_dim = len(FEATURE_COLUMNS)
    FINAL_MODEL = LSTMPredictor(
        input_dim=input_dim,
        hidden_dim=128,
        num_layers=2,
        dropout=0.3
    ).to(DEVICE)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(FINAL_MODEL.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    
    num_epochs = 100
    best_loss = float('inf')
    patience = 15
    patience_counter = 0
    
    print("Starting training...")
    FINAL_MODEL.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            
            predictions = FINAL_MODEL(batch_X)
            
            loss = criterion(predictions, batch_y)
            
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(FINAL_MODEL.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        scheduler.step(avg_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}")
        
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    FINAL_MODEL.eval()
    print(f"Model training complete! Best Loss: {best_loss:.6f}")
    
    buffer_size = SEQUENCE_LENGTH + 50
    HISTORY_BUFFER = training_df.select(
        ['date_id'] + FEATURE_COLUMNS + ['target', 'forward_returns', 'risk_free_rate']
    ).tail(buffer_size)
    
    elapsed = (pd.Timestamp.now() - start_time).total_seconds()
    print(f"Model training complete in {elapsed:.1f} seconds. Ready for predictions.")

def predict(test: pl.DataFrame) -> float:
    """
    Make a prediction for a single test row.
    
    Args:
        test: A polars DataFrame with one row containing the test features
        
    Returns:
        A float signal value between 0 and 2
    """
    global HISTORY_BUFFER, FINAL_MODEL, SCALER, FEATURE_COLUMNS, SEQUENCE_LENGTH
    
    try:
        feature_cols = [col for col in test.columns if col != 'date_id']
        test = test.with_columns(pl.col(feature_cols).cast(pl.Float64, strict=False))
        
        rename_mapping = {}
        if 'lagged_forward_returns' in test.columns:
            rename_mapping['lagged_forward_returns'] = 'forward_returns'
        if 'lagged_risk_free_rate' in test.columns:
            rename_mapping['lagged_risk_free_rate'] = 'risk_free_rate'
        if 'lagged_market_forward_excess_returns' in test.columns:
            rename_mapping['lagged_market_forward_excess_returns'] = 'target'
        
        if rename_mapping:
            test = test.rename(rename_mapping)
        
        if 'is_scored' in test.columns:
            test = test.drop('is_scored')
        
        for col in HISTORY_BUFFER.columns:
            if col not in test.columns:
                test = test.with_columns(pl.lit(0.0).cast(pl.Float64).alias(col))
        
        test = test.select(HISTORY_BUFFER.columns)
        
        HISTORY_BUFFER = pl.concat([HISTORY_BUFFER, test], how="vertical")
        
        max_buffer = SEQUENCE_LENGTH + 100
        if len(HISTORY_BUFFER) > max_buffer:
            HISTORY_BUFFER = HISTORY_BUFFER.tail(max_buffer)
        
        if len(HISTORY_BUFFER) < SEQUENCE_LENGTH:
            needed = SEQUENCE_LENGTH - len(HISTORY_BUFFER)
            first_row = HISTORY_BUFFER.head(1)
            padding = pl.concat([first_row] * needed, how="vertical")
            sequence_data = pl.concat([padding, HISTORY_BUFFER], how="vertical")
        else:
            sequence_data = HISTORY_BUFFER.tail(SEQUENCE_LENGTH)
        
        X_sequence = sequence_data.select(FEATURE_COLUMNS).to_numpy()
        
        X_sequence = np.nan_to_num(X_sequence, nan=0.0)
        
        X_sequence_scaled = SCALER.transform(X_sequence)
        
        X_sequence_scaled = X_sequence_scaled.reshape(1, SEQUENCE_LENGTH, -1)
        
        X_tensor = torch.FloatTensor(X_sequence_scaled).to(DEVICE)
        
        with torch.no_grad():
            raw_prediction = FINAL_MODEL(X_tensor).cpu().numpy()[0, 0]
        
        signal = convert_to_signal(np.array([raw_prediction]))[0]
        
        return float(signal)
        
    except Exception as e:
        print(f"Error in prediction: {e}")
        import traceback
        traceback.print_exc()
        return 1.0

print("="*50)
print("Starting LSTM model training...")
print("="*50)
train_model()
print("="*50)
print("Model ready. Starting inference server...")
print("="*50)

In [ ]:
import kaggle_evaluation.default_inference_server

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("Running in competition mode")
    inference_server.serve()
else:
    print("Running in local gateway mode")
    inference_server.run_local_gateway(
        ('/kaggle/input/hull-tactical-market-prediction/',)
    )